In [5]:
import torch
import uuid
import os

B = 4
N = 5
af = torch.rand(B, N, 512)
af = af.view(B * N, 512)
ap = torch.rand(B, 4, 4).unsqueeze(1).repeat(1, N, 1, 1)
ap = ap.view(B * N, 4, 4)
ap2ds = torch.rand(B, N, 2).view(B * N, 2)

bf = torch.rand(B, N, 512).view(B * N, 512)
bp = torch.rand(B, 4, 4).unsqueeze(1).repeat(1, N, 1, 1).view(B * N, 4, 4)
bp2ds = torch.rand(B, N, 2).view(B * N, 2)

f = (af + bf) / 2

f = f.tolist()

p3ds = torch.rand(B, N, 3).view(B * N, 3).tolist()


feature_ids = [uuid.uuid4().hex for _ in range(B * N)]

keyframe_a_ids = [uuid.uuid4().hex for _ in range(B)]
keyframe_b_ids = [uuid.uuid4().hex for _ in range(B)]

keypoint_id_a_lists = [[uuid.uuid4().hex for _ in range(N)] for _ in range(B)]
keypoint_id_b_lists = [[uuid.uuid4().hex for _ in range(N)] for _ in range(B)]

keypoint_id_lists = keypoint_id_a_lists + keypoint_id_b_lists

keypoint_a_ids = [k for ks in keypoint_id_a_lists for k in ks]
keypoint_b_ids = [k for ks in keypoint_id_b_lists for k in ks]

keypoint_ids = keypoint_a_ids + keypoint_b_ids

landmark_id_lists = [[uuid.uuid4().hex for _ in range(N)] for _ in range(B)]
landmark_ids = [l for ls in landmark_id_lists for l in ls]
key_landmark_ids = landmark_id_lists + landmark_id_lists


params=dict(
    feature_ids=feature_ids,
    features=f,
)

In [1]:
from pycozo import Client
client = Client('rocksdb', 'map.db')
client.run(open('init.DIALOG').read())
client.run(open('index.DIALOG').read())

,status
0,OK


In [27]:
def rule_zip_impl(inputs, options):
    x, y = inputs
    x = [j for i in x for j in i]
    y = [j for i in y for j in i]
    return list(zip(x, y))
client.unregister_fixed_rule('Zip')
client.register_fixed_rule('Zip', 2, rule_zip_impl)

In [28]:
CMD = """
{   
    r_ids[] <- [[$feature_ids]]
    r_data[] <- [[$features]]
    ?[feature_id, feature] <~ Zip(r_ids[], r_data[])
}
"""
client.run(CMD, params=params)

,feature_id,feature
0,"[86f570fbff6a4012baa909491f7475d1, bbeb8141316...","[[0.23952218890190125, 0.4931360185146332, 0.3..."


In [12]:
%cozo_run_file 'update.DIALOG'

AttributeError: 'str' object has no attribute 'get'

In [5]:
%cozo_run_string "?[feature_id, data] := *feature[feature_id, data]"

,feature_id,data
0,aa4f0b31-952b-4302-9dd2-7f6efb94a9f7,"[0.985819935798645, 0.18311095237731934, 0.390..."
1,839d8cac-0c4d-4362-a798-5ae107701d96,"[0.985819935798645, 0.18311095237731934, 0.390..."
2,24881a4b-6418-45e6-bdcf-991175c475fe,"[0.985819935798645, 0.18311095237731934, 0.390..."
3,79d8d11d-2677-4782-a692-c8ac73fafb7f,"[0.985819935798645, 0.18311095237731934, 0.390..."
4,f8b21c86-6e50-4b78-9f11-0557dac7082c,"[0.985819935798645, 0.18311095237731934, 0.390..."
5,39d79d5d-e2f0-4df0-b241-df9729d01201,"[0.985819935798645, 0.18311095237731934, 0.390..."
6,f50f248d-cf10-4e6f-8c61-69167f7b4d51,"[0.985819935798645, 0.18311095237731934, 0.390..."
7,6c461828-d2f1-4f47-be34-911d016a588e,"[0.985819935798645, 0.18311095237731934, 0.390..."


In [19]:
%cozo_run_string "?[landmark_id, xyz, feature, keypoint_ids] := *landmark[landmark_id, xyz, feature, keypoint_ids]"

,landmark_id,xyz,feature,keypoint_ids


In [6]:
%cozo_run_string "?[keyframe_id, pose, keypoint_ids] := *keyframe[keyframe_id, pose, keypoint_ids]"

,keyframe_id,pose,keypoint_ids


In [5]:
%cozo_run_string "?[keypoint_id, xy, keyframe_id, landmark_id] := *keypoint[keypoint_id, xy, keyframe_id, landmark_id]"

,keypoint_id,xy,keyframe_id,landmark_id
